# Object Relation Mapping of a Fitbit Dataset 

### Importing libraries and define file paths

In [2]:
from pandas import read_json  as json_to_table
from pandas import read_excel as excel_to_table
from pandas import read_csv as csv_to_table
from pony.orm import Database, Required, Optional, PrimaryKey, Set, select, avg, count
from datetime import datetime, timedelta
from pandas import DataFrame  as Table
from pandas import merge as Merge_df
from pandas import date_range, concat
from functools import reduce
import jupyter_dash
from dash import html
import dash_pivottable
import dash
import pandas as pd

import logging

In [3]:
# #File path definition LINDA
# PARTECIPANTS_FILE = 'participant-overview.xlsx' 
# PMDATA = 'pmdata\\'
# CALORIES = '\\fitbit\calories.json'
# DISTANCE = '\\fitbit\distance.json'
# EXERCISE = '\\fitbit\exercise.json'
# HEART_RATE = '\\fitbit\heart_rate.json'
# LIGHTLY_ACTIVE_MIN = '\\fitbit\lightly_active_minutes.json'
# MODERATE_ACTIVE_MIN = '\\fitbit\moderately_active_minutes.json'
# RESTING_HEART = '\\fitbit\\resting_heart_rate.json'
# SEDENTARY_MIN = '\\fitbit\sedentary_minutes.json'
# SLEEP_SCORE = '\\fitbit\sleep_score.csv'
# SLEEP = '\\fitbit\sleep.json'
# STEPS = '\\fitbit\steps.json'
# TIME_HEART_ZONE_RATE = '\\fitbit\\time_in_heart_rate_zones.json'
# VERY_ACTIVE_MIN = '\\fitbit\\very_active_minutes.json'
# PATH_LIST_JSON = [CALORIES, DISTANCE, EXERCISE, HEART_RATE, LIGHTLY_ACTIVE_MIN, 
#         MODERATE_ACTIVE_MIN, RESTING_HEART, SEDENTARY_MIN, SEDENTARY_MIN,
#         SLEEP, STEPS, TIME_HEART_ZONE_RATE, VERY_ACTIVE_MIN]
# LOG = '../1.Data Warehouse OLAP systems for healthcare/'

#File path definition ALBE
PARTECIPANTS_FILE = 'participant-overview.xlsx' 
PMDATA = 'pmdata\\'
CALORIES = '\\fitbit\calories.json'
DISTANCE = '\\fitbit\distance.json'
EXERCISE = '\\fitbit\exercise.json'
HEART_RATE = '\\fitbit\heart_rate.json'
LIGHTLY_ACTIVE_MIN = '\\fitbit\lightly_active_minutes.json'
MODERATE_ACTIVE_MIN = '\\fitbit\moderately_active_minutes.json'
RESTING_HEART = '\\fitbit\\resting_heart_rate.json'
SEDENTARY_MIN = '\\fitbit\sedentary_minutes.json'
SLEEP_SCORE = '\\fitbit\sleep_score.csv'
SLEEP = '\\fitbit\sleep.json'
STEPS = '\\fitbit\steps.json'
TIME_HEART_ZONE_RATE = '\\fitbit\\time_in_heart_rate_zones.json'
VERY_ACTIVE_MIN = '\\fitbit\\very_active_minutes.json'
PATH_LIST_JSON = [CALORIES, DISTANCE, EXERCISE, HEART_RATE, LIGHTLY_ACTIVE_MIN, 
        MODERATE_ACTIVE_MIN, RESTING_HEART, SEDENTARY_MIN, SEDENTARY_MIN,
        SLEEP, STEPS, TIME_HEART_ZONE_RATE, VERY_ACTIVE_MIN]
LOG = '../1.Data Warehouse OLAP systems for healthcare/'
#

## DataBase 

In [4]:
#Connessione al DB !RIcordarsi di far andare il server!
DB = Database()
DB.bind(provider='postgres', 
        database='postgres',
        user='postgres', 
        port='5433',  #Linda: 5432
        host='localhost',
        password='admin'
        )

## LogDebug

In [5]:
from datetime import datetime as a
log_date_filename = 'HIS_'+a.today().strftime('%Y%m%d_%H%M%S')+'.log'
log_date_filename

logging.basicConfig(
                    filename = log_date_filename
                    , filemode='w'
                    , format = "[%(asctime)s] %(levelname)s - %(message)s"
                    , level = logging.DEBUG
                    , force = True
            )
#basicConfig setta le configuarioni base del file dove salvare i log del codice
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)  #setta il livello di logging 

logger.debug("********* INIZIO LOG *********")


### Table creation

**PARTECIPANT**

In [6]:

#Si va a creare la classe Partecipant che corrisponde alla tabella a db. 
class Partecipant(DB.Entity):
    name = PrimaryKey(str)
    age  = Optional(int)
    height = Optional(int)
    gender = Optional(str)
    max_heart_rate = Optional(float)
    group = Optional(str)
    base_line_cal = Optional(float)
    BPM = Set('BPM')
    calories = Set('Calories')
    exercise = Set('Exercises')
    sleep = Set('Sleep')
    steps = Set('Steps')
    distance = Set('Distances')
    day = Set('Day')

#Inizio implememtazione delle funzioni che verranno utilizzate poi per elaborare i dati


     # Controlla la presenza dei partecipanti in un specifico giorno.
    @classmethod
    def add_partecipant_day(self, name, day):
        try:
            logger.debug('Starting method add_partecipant_day')
            if not self.checkPartecipantExists(name):  #se non esiste richiama la funzione che permette di cresre un nuovo partecipante
                print(f'Partecipant {name} does not exists')
                print('Starting creation new partecipant')
                self.create_new_partecipant(name)
            else:
                print(f'Partecipant already exists, go to creation of tables for day ')
                
            
            day = datetime.strptime(day, "%Y-%m-%d") #cast che trasforma da stringa a datetime


            # Creazione funzioni che importano i dati nel DB
            #
            # Calories
            self.createCalories(date = day, file = json_to_table("".join([PMDATA, name, CALORIES])), name = name)
            # Distances
            self.createDistances(date = day, file = json_to_table("".join([PMDATA, name, DISTANCE])), name = name)
            # BPM
            self.createBPM(date = day, file = json_to_table("".join([PMDATA, name, HEART_RATE])), name = name)
            # Exercise
            self.createExercise(date = day, file = json_to_table("".join([PMDATA, name, EXERCISE])), name = name)
            # Sleep
            self.createSleep(date = day, file = json_to_table("".join([PMDATA, name, SLEEP])), name = name)
            # StepsSLEEP
            self.createSteps(date = day, file = json_to_table("".join([PMDATA, name, STEPS])), name = name)
            # Day
            self.createDay(date = day, name = name)

            DB.commit()
            
        except IndexError:
            # This block will handle IndexError specifically
            print("IndexError occurred in day. The index is out of range, but the program will continue.")
            DB.commit()
        except Exception as e:
            # This block will handle all other exceptions
            print(f"An unexpected error occurred: {e}")
            DB.rollback()
        finally:
            logger.debug(f'Fine della scrittura del log')
            logging.shutdown()

    #Funzione per creare un nuovo partecipanete se non esiste già
    @classmethod
    def create_new_partecipant(self, name):
        print('Start creation new partecipant')
        logger.debug('Inizio metodo create_new_partecipant()')
        logger.debug('Lettura file excel e salvataggio in df')
        df = excel_to_table(PARTECIPANTS_FILE, header=1, engine='openpyxl')  #permette di leggere il file excel e salvarlo in un dataframe
        
        logger.debug('Lettura singoli dati per colonna')
        #Si scorre e si estraggono i dati dal dataframe appena creato, a partire dal nome del partecipante
        age = (list(df[ df["Partecipant ID"] == name].Age)[0])
        logger.debug("Age = %s",age, exc_info=True)
        
        height =  list(df[ df["Partecipant ID"]  == name].Height)[0]
        logger.debug("Height = %s",height, exc_info=1)

        gender = list(df[ df["Partecipant ID"]  == name].Gender)[0]
        logger.debug("gender = %s",gender, exc_info=1)
        max_heart_rate = (list(df[ df["Partecipant ID"]  == name].Max_heart_rate)[0])
        if max_heart_rate is None:
            max_heart_rate = 0
        logger.debug("max_heart_rate = %s",{max_heart_rate}, exc_info=1)
        group = (list(df[ df["Partecipant ID"]  == name].A_or_B_person))[0]
        logger.debug("group = %s",group, exc_info=1)

        partecipant = Partecipant(
            name = name,
            age = age,
            height = height,
            gender = gender,
            max_heart_rate = max_heart_rate,
            group = group
        )
        
        return partecipant


    #Funzione che va ad inserire tutute le calories del giorno passato in input
    @classmethod
    def createCalories(self, date, file, name):
        print('Start creation calories table')
        #date  = datetime.strptime(date, "%Y-%m-%d")  NEcessario per le singole chiamate se si vogliono testare
        print(date)
        #Funzione che ritorna una lista di valori filtrati per data. 
        a = filter(lambda x:
                (x >= date and x < datetime(date.year, date.month, date.day+1)),
                list(file.dateTime)
                )
        print(a)
        list_filtered_date = list(a)
        print(list_filtered_date)
        base_cal = min(file.value) #si va a prendere il min valore delle calorie presente nel file
        print(base_cal)
        # p = Partecipant(name = name) #csi crea un istanza di tipo Partecipant utilizzando il nome passato in inpurt
        # print(p)
        Partecipant.base_line_cal = base_cal

        for date in list_filtered_date:
            if float(list(file[ file["dateTime"] == date].value)[0]) > base_cal:  #Se il valore delle calorie per la data è maggiore della base cal
                Calories(partecipant = name,   #si crea un istanza di calories con il nome, data e e il valore della caloria filtrato
                        vt = date,
                        value = float(list(file[ file["dateTime"] == date].value)[0])
                )


   #Funzione che va a inserire e creare tutte le distanze della data passata in input
    @classmethod
    def createDistances(self, date, file, name):
        print('Start creation distances table')
        a = filter(lambda x:
                   (x >= date and x < datetime(date.year, date.month, date.day+1)),
                   list(file.dateTime)
                   )
        list_filtered_date = list(a)
        
        for date in list_filtered_date:
            Distances(partecipant = name,
                    value = float(list(file[ file["dateTime"] == date].value)[0]),
                    vt = date
            )

#Funzione che va a inserire e creare tutti i valori di BPM  della data passata in input
    @classmethod
    def createBPM(self, date, file, name): 
        print('Start creation bpm table')
        a = filter(lambda x:
                   (x >= date and x < datetime(date.year, date.month, date.day+1)),
                   list(file.dateTime)
                   )
        list_filtered_date = list(a)
        
        for date in list_filtered_date:
            BPM(partecipant = name, 
                value = list(file[ file["dateTime"] == date].value)[0].get("bpm"),
                vt = date, 
                confidence = list(file[ file["dateTime"] == date].value)[0].get("confidence")
                )
    
    @classmethod
    def createExercise(self, date, file, name):
        print('Start creation exercises table')
        #iterazione per ogni valore nella colonna facedno il cast per trasforamre da stringa a datetime. I risultati vengono salvati una lista
        list_day = [datetime.strptime(day, "%Y-%m-%d %H:%M:%S") for day in list(file.startTime)]
        #function that return a list of values filtered by date 
        a = filter(lambda x: 
            (x >= date and x< datetime(date.year, date.month, date.day+1)) , 
            list_day)
        lista_date_filtrate = list(a)
       
         #loop lista date creata sopra. dove viene ti trasforma il datetime in stringa e si va a crare l'istanza Exercises pasadno 
         #i valori letti dal file in base alla data filtrata sopra.
        for date in lista_date_filtrate:
            date = date.strftime("%Y-%m-%d %H:%M:%S")   
            Exercises(
                vt = date,
                partecipant = name,
                activityName = list(file[ file["startTime"] == date].activityName)[0],
                avg_BPM = list(file[ file["startTime"] == date].averageHeartRate)[0],
                calories = list(file[ file["startTime"] == date].calories)[0],
                duration = list(file[ file["startTime"] == date].duration)[0],
                steps = list(file[ file["startTime"] == date].steps)[0],
                elevationGain = list(file[ file["startTime"] == date].elevationGain)[0]
            )

    @classmethod
    def createSleep(self, date, file, name):
        print('Start creation sleep table')
        list_startTime = [datetime.strptime(day, "%Y-%m-%d %H:%M:%S") for day in list(file.startTime)]

        #for date in list_startTime:         
        a = filter(lambda x:
                   (x >= date and x < datetime(date.year, date.month, date.day+1)),
                   list_startTime
                )
        list_filtered_date = list(a)
      #  print(list_filtered_date)

        for date in list_filtered_date:
            date = datetime.strftime(date, "%Y-%m-%d %H:%M:%S")
            
            Sleep(
                vt = date,
                partecipant = name,
                duration = list(file[ file["startTime"] == date].duration)[0]/60000,
                endTime = list(file[ file["startTime"] == date].endTime)[0],
                minutesAsleep = list(file[ file["startTime"] == date].minutesAsleep)[0],
                types = list(file[ file["startTime"] == date].type)[0],
                efficiency = list(file[ file["startTime"] == date].efficiency)[0]
            )

    @classmethod
    def createSteps(self, date, file, name):
        print('Start creation steps table')
        # function that create/insert BPM of the specific day
        a = filter(lambda x:
                   (x >= date and x < datetime(date.year, date.month, date.day+1)),
                   list(file.dateTime)
                   )
        list_filtered_date = list(a)
        
        for date in list_filtered_date:
            Steps(
                partecipant = name,
                value = list(file[ file["dateTime"] == date].value)[0],
                vt = date
            )

    @classmethod
    def createDay(self, name, date):
        print('Start creation day table')
        lightly_active_minutes_file =  json_to_table("".join([PMDATA, name, LIGHTLY_ACTIVE_MIN]))
        moderately_active_minutes_file = json_to_table("".join([PMDATA, name, MODERATE_ACTIVE_MIN]))
        resting_heart_rate_file = json_to_table("".join([PMDATA, name, RESTING_HEART]))
        sedentary_minutes_file = json_to_table("".join([PMDATA, name, SEDENTARY_MIN]))
        very_active_minutes_file = json_to_table("".join([PMDATA, name, VERY_ACTIVE_MIN]))
        time_in_heart_rate_zones_file = json_to_table("".join([PMDATA, name, TIME_HEART_ZONE_RATE]))
        sleep_score_file = csv_to_table("".join([PMDATA, name, SLEEP_SCORE]))
        IndexDay = [ list(sleep_score_file[sleep_score_file['timestamp'] == i].index.values)[0] for i in sleep_score_file['timestamp'] if datetime.strptime(i[:-10], "%Y-%m-%d") == date ]
    #    'overall_score', 'composition_score',
    #    'revitalization_score', 'duration_score', 'deep_sleep_in_minutes',
    #    'resting_heart_rate', 'restlessness']
      #  w= float(list(file[ file["dateTime"] == date].value)[0])

        print(IndexDay)

        try:
            n_overall_score = sleep_score_file.loc[IndexDay[0]].overall_score
            n_composition_score = sleep_score_file.loc[IndexDay[0]].composition_score
            n_revitalization_score = sleep_score_file.loc[IndexDay[0]].revitalization_score
            n_duration_score = sleep_score_file.loc[IndexDay[0]].revitalization_score
            n_deep_sleep_in_minutes = sleep_score_file.loc[IndexDay[0]].deep_sleep_in_minutes
        except IndexError:
            n_overall_score = None
            n_composition_score = None
            n_revitalization_score = None
            n_duration_score = None
            n_deep_sleep_in_minutes = None
        finally:
            Day(

                partecipant = name,
                vt = date,
                n_lightly_active_minutes = list(lightly_active_minutes_file[ lightly_active_minutes_file["dateTime"] == date].value)[0],
                n_moderately_active_minutes = list(moderately_active_minutes_file[ moderately_active_minutes_file["dateTime"] == date].value)[0],
                n_sedentary_minutes = list(sedentary_minutes_file[ sedentary_minutes_file["dateTime"] == date].value)[0],
                n_very_active_minutes = list(very_active_minutes_file[ very_active_minutes_file["dateTime"] == date].value)[0],
                n_overall_score = n_overall_score,
                n_composition_score = n_composition_score,
                n_revitalization_score = n_revitalization_score,
                n_duration_score = n_duration_score,
                n_deep_sleep_in_minutes = n_deep_sleep_in_minutes,
                f_resting_heart_rate = list(resting_heart_rate_file[ resting_heart_rate_file["dateTime"] == date].value)[0].get("value"),
                #use .get to access to children attributes of the xml
                f_time_in_heart_rate_BELOW_DEFAULT_ZONE_1 = list(time_in_heart_rate_zones_file[ time_in_heart_rate_zones_file["dateTime"] == date].value)[0].get("valuesInZones").get("BELOW_DEFAULT_ZONE_1"),
                f_time_in_heart_rate_zone_1 = list(time_in_heart_rate_zones_file[ time_in_heart_rate_zones_file["dateTime"] == date].value)[0].get("valuesInZones").get("IND_DEFAULT_ZONE_1"),
                f_time_in_heart_rate_zone_2 = list(time_in_heart_rate_zones_file[ time_in_heart_rate_zones_file["dateTime"] == date].value)[0].get("valuesInZones").get("IND_DEFAULT_ZONE_2"),
                f_time_in_heart_rate_zone_3 = list(time_in_heart_rate_zones_file[ time_in_heart_rate_zones_file["dateTime"] == date].value)[0].get("valuesInZones").get("IND_DEFAULT_ZONE_3")
            )
           

 
    @classmethod
    def checkPartecipantExists(self, name):
        return not(Partecipant.get(name = name) == None) 
    
    @classmethod
    def checkExistsDate(self, part, date):
       # print('ExistDate:')
       # print(Day.get(vt = date, partecipant = part))
        return not(Day.get(vt = date, partecipant = part) == None)
    
    @classmethod
    def time_Aggregation(self, vts, vte, minutes):
        vts = datetime.strptime(vts, "%Y-%m-%d")
        vte = datetime.strptime(vte, "%Y-%m-%d")
        # calculate total minutes in a day and divide them by the desired minutes
        num_rows = int((24*60)/minutes)
        print(num_rows, vts, vte)
        partecipants = list(select(p.name for p in Partecipant))
        print(partecipants)
        partec2 =[]
        #print(date_range(start = vts, end = vte).to_datetime().tolist())
        # check for the presence in the db of the data. If not data will be loaded to db
        for  day in date_range(start = vts, end = vte): #.to_datetime().tolist():
            for p in partecipants:
                print('Day for: ', day)
                if self.checkExistsDate(part = p, date = day):
                    print(self.checkExistsDate(part = p, date = day))
                    print(day, ' day not exist in db', p)
                    partec2.append(p)
        print(partec2)

        df = Table([])

        if partec2 != []:
            for p in partec2:
                print('p2: ', p)
                df_BPM = self.create_df_timeAggregation_BPM2( part = p, min_aggregation = num_rows, vts = vts,  vte = vte )
                df_calo = self.create_df_timeAggregation_Calories( part = p, min_aggregation = num_rows, vts = vts,  vte = vte )
                df_dis = self.create_df_timeAggregation_Distances( part = p, min_aggregation = num_rows, vts = vts,  vte = vte )
                df_st = self.create_df_timeAggregation_Steps( part = p, min_aggregation = num_rows, vts = vts,  vte = vte )
                df_exer = self.create_df_timeAggregation_Exercise( part = p, min_aggregation = num_rows, vts = vts,  vte = vte )
                 
                list_df = [df_BPM, df_calo, df_dis, df_st, df_exer]  
                print(list_df)          
                list_df = reduce(lambda  left,right:   #reduce funcction concatena iterativamente in modo orizzontale i dataframe richiamando la funzionù
                                 #concat. Questo vieen specificato mettendo axis = 1 che specifica che la concatenazione avviene per colonne.
                                 concat([left,right], axis=1), list_df
                                 ).fillna('no value') #utilizzato per per sostituire i valori mancantin con la dicitura 'no value'
                   
                df = concat([df, list_df]) #concatena i due dataframe verticalmente. Si vanno quindi ad aggiungere alle righe di df le righe di list_dfù
                print(df)



#Funzione per creare un dataframe che aggrega i dati di BPM per un dato intervallo temporale
    @classmethod
    def create_df_timeAggregation_BPM2(cls, part, min_aggregation, vts, vte):
        date_list = pd.date_range(start=vts, end=vte, freq='Min').tolist()
        df = []

        v = select((bpm.vt, bpm.value) 
               for bpm in BPM 
               if (bpm.vt >= vts and bpm.vt <= vte and part == str(bpm.partecipant.name))
               ).order_by(1)         
        
        v = Table(v, columns=['vt', 'value'])
    
        interval = timedelta(minutes=min_aggregation)  # Intervallo di tempo per il raggruppamento
    
        start_time = vts
        end_time = start_time + interval
    
        while end_time <= vte:
            subset = v[(v['vt'] >= start_time) & (v['vt'] < end_time)]
  
            mean_value = int(subset['value'].mean())
            df.append([mean_value, part, start_time, end_time])
        
            start_time = end_time
            end_time = start_time + interval
    
        if start_time < vte:
            subset = v[v['vt'] >= start_time]
            mean_value = int(subset['value'].mean())
            df.append([mean_value, part, start_time, vte])

        df = pd.DataFrame(df, columns=['heartRate', 'Partecipant', 'StartTime', 'EndTime'])
        return df


    @classmethod
    def create_df_timeAggregation_Calories( self, part, min_aggregation, vts, vte):

        df2 = []
        # prende tutti gli elementi tra vts (data start) and vte (data end) e gli ordina in base alla data
        v = select((cal.vt, cal.value) 
                   for cal in Calories 
                   if (cal.vt >= vts and cal.vt <= vte and part == str(cal.partecipant.name))
                   ).order_by(1) 
        
        v = Table(v, columns = ['vt', 'value'])   
        #print(v)
        interval = timedelta(minutes=min_aggregation)  # Intervallo di tempo per il raggruppamento
    
        start_time = vts
        end_time = start_time + interval   
        #print(end_time)  
        while end_time <= vte:
            subset = v[(v['vt'] >= start_time) & (v['vt'] < end_time)]
            sum_value = int(subset['value'].sum())
            df2.append([sum_value, part, start_time, end_time])
        
            start_time = end_time
            end_time = start_time + interval
    
        if start_time < vte:
            subset = v[v['vt'] >= start_time]
            sum_value = int(subset['value'].sum())
            df2.append([sum_value, part, start_time, vte])

        df2 = pd.DataFrame(df2, columns=['Calories', 'Partecipant', 'StartTime', 'EndTime'])
        return df2
    
    @classmethod
    def create_df_timeAggregation_Distances( self, part, min_aggregation, vts, vte):
       #df = [] 
       # prende tutti gli elementi tra vts (data start) and vte (data end) e gli ordina in base alla data
        v = select((d.vt, d.value) 
                   for d in Distances 
                   if (d.vt >= vts and d.vt <= vte and part == str(d.partecipant.name))
                   ).order_by(1) 
        
        v = Table(v, columns = ['validTime', 'value'])

        df2 = [] 
        interval = timedelta(minutes=min_aggregation)  # Intervallo di tempo per il raggruppamento
    
        start_time = vts
        end_time = start_time + interval   
        #print(end_time)  
        while end_time <= vte:
            subset = v[(v['validTime'] >= start_time) & (v['validTime'] < end_time)]
            sum_value = int(subset['value'].sum())
            df2.append([sum_value, part, start_time, end_time])
        
            start_time = end_time
            end_time = start_time + interval
    
        if start_time < vte:
            subset = v[v['validTime'] >= start_time]
            sum_value = int(subset['value'].sum())
            df2.append([sum_value, part, start_time, vte])

        df2 = pd.DataFrame(df2, columns=['Distances', 'Partecipant', 'StartTime', 'EndTime'])

        return df2
    
    
    @classmethod
    def create_df_timeAggregation_Steps( self, part, min_aggregation, vts, vte):
        # prende tutti gli elementi tra vts (data start) and vte (data end) e gli ordina in base alla data
        v = select((s.vt, s.value) 
                   for s in Steps 
                   if (s.vt >= vts and s.vt <= vte and part == str(s.partecipant.name))
                   ).order_by(1)         
        
        v = Table(v, columns = ['validTime', 'value'])       
    
        df2 = [] 
        interval = timedelta(minutes=min_aggregation)  # Intervallo di tempo per il raggruppamento
    
        start_time = vts
        end_time = start_time + interval   
        #print(end_time)  
        while end_time <= vte:
            subset = v[(v['validTime'] >= start_time) & (v['validTime'] < end_time)]
            sum_value = int(subset['value'].sum())
            df2.append([sum_value, part, start_time, end_time])
        
            start_time = end_time
            end_time = start_time + interval
    
        if start_time < vte:
            subset = v[v['validTime'] >= start_time]
            sum_value = int(subset['value'].sum())
            df2.append([sum_value, part, start_time, vte])

        df2 = pd.DataFrame(df2, columns=['Steps', 'Partecipant', 'StartTime', 'EndTime'])

        return df2
    
    @classmethod
    def create_df_timeAggregation_Exercise( self, part, min_aggregation, vts, vte):
    
        date_list = date_range(start=vts,end=vte, freq='Min').to_pydatetime().tolist()#date_range è una funzione pandas che va a crare un range temporale a partire da
                     #le validtimestart e valide time end con frequenza di 1 minuto. to_pydatetime è per converitre il risultato in una lista python

        # print(date_list)
        df = Table([day for day in date_list], columns= ['validTime']) 
        # print(df, len(df)-1)

        tmp_list = []
        for i in range(0, len(df)-1, min_aggregation):         
            tmp_list.append([ df['validTime'].loc[i], False, False, False, False, False, False, df['validTime'].loc[i] + timedelta(minutes=min_aggregation) ])

        # print(tmp_list)
        df = Table(tmp_list, columns=['validTime', 'walking', 'swimming', 'running', 'cycling', 'treadmill' , 'workout' , 'endVt'])       
        
        #take all the element between the vts and vte  sort the data in order of date
        v = select((e.vt, e.duration ,e.activityName) 
                   for e in Exercises
                     if (e.vt >= vts and e.vt <= vte and part == str(e.partecipant.name))
                    ).order_by(1)         
        v = Table(v, columns = ['validTime', 'duration', 'activityname'])

        v1 = Table(v, columns= ['validTime', 'duration', 'activityname' , 'endTime', 'cicles'] )
        v1['endTime'] = '00-00-00 00:00:00'
        v1['cicles'] = 0
        # print(v1)

        # for a in v1:
        #     print(a)
        #     print(v1[a])
        #     # print(v1['validTime'].loc[a])
        #     # v1['endTime'].loc[a] = v1['validTime'].loc[a] + timedelta(milliseconds = v1['duration'].loc[a])
        
        
        for g in range(0, len(v1)): 
            v1['endTime'].loc[g] = v1['validTime'].loc[g] + timedelta(milliseconds = int(v1['duration'].loc[g]))
            print(round((v1['endTime'].loc[g] - v1['validTime'].loc[g]).seconds / 60 / min_aggregation)) 
            print((v1['endTime'].loc[g] - v1['validTime'].loc[g]).seconds / 60 / min_aggregation)
            b = v1['validTime'].loc[g]
            print(b)
            while b + timedelta(minutes = min_aggregation) <= v1['endTime'].loc[g]:
                v1['cicles'].loc[g] = v1['cicles'].loc[g] + 1
                b = b + timedelta(minutes = min_aggregation)

        print(df)
        print(v1)

    
        for i in range(0, len(v1)): 
            for j in range(0, len(df)-1):
                if v1['validTime'].loc[i] >= df['validTime'].loc[j] and v1['validTime'].loc[i] <= df["validTime"].loc[j+1] :
                    for cicle in range(0,(v1['cicles'].loc[i] + 1)):
                        # print(i,j)
                        if str(v1['activityname'].loc[i]) == 'Walk':
                            df['walking'].loc[j+cicle] = True
                        elif str(v1['activityname'].loc[i]) == 'Swim' :
                            df['swimming'].loc[j+cicle] = True
                        elif str(v1['activityname'].loc[i])  == 'Run' :
                            df['running'].loc[j+cicle] = True
                        elif str(v1['activityname'].loc[i])  == 'Cycle' :
                            df['cycling'].loc[j+cicle] = True
                        elif str(v1['activityname'].loc[i]) == 'Treadmill':
                            df['treadmill'].loc[j+cicle] = True
                        elif str(v1['activityname'].loc[i]) == 'Workout':
                            df['workout'].loc[j+cicle] = True
        
        print(df)
                    
        return df



class Calories(DB.Entity):
    value = Required(float)
    vt = Required(datetime)
    partecipant = Required(Partecipant)
    PrimaryKey(partecipant, vt)

class Distances(DB.Entity):
    value = Required(float)
    vt = Required(datetime)
    partecipant = Required(Partecipant)
    PrimaryKey(vt, partecipant)

class BPM(DB.Entity):
    vt = Required(datetime)
    value = Optional(int)
    confidence = Optional(int)
    partecipant = Required(Partecipant)
    PrimaryKey(partecipant, vt)

class Exercises(DB.Entity):
    vt = Required(datetime)
    partecipant = Required(Partecipant)
    # attributes of the json
    activityName = Required(str)
    avg_BPM = Optional(float)
    calories = Optional(int)
    duration = Optional(int)
    steps = Optional(float)
    elevationGain = Optional(float)
    PrimaryKey(vt, partecipant)

class Sleep(DB.Entity):
    vt = Required(datetime)
    partecipant = Required(Partecipant)
    # attributes of the json
    duration = Optional(float)
    endTime = Optional(datetime)
    minutesAsleep = Optional(int)
    types = Optional(str)
    efficiency = Optional(int)
    PrimaryKey(partecipant, vt)

class Steps(DB.Entity):
    vt = Required(datetime)
    partecipant = Required(Partecipant)
    value = Required(int)   
    PrimaryKey(vt, partecipant) 


class Day(DB.Entity):
    vt = Required(datetime)
    partecipant = Required(Partecipant)
    PrimaryKey(partecipant, vt)
    n_lightly_active_minutes = Optional(int)
    n_moderately_active_minutes = Optional(int)
    f_resting_heart_rate = Optional(float)
    n_sedentary_minutes = Optional(int)
    n_very_active_minutes = Optional(int)
    n_overall_score = Optional(int)
    n_composition_score = Optional(int)
    n_revitalization_score = Optional(int)
    n_duration_score = Optional(int)
    n_deep_sleep_in_minutes = Optional(int)
    f_time_in_heart_rate_BELOW_DEFAULT_ZONE_1 = Optional(float)
    f_time_in_heart_rate_zone_1 = Optional(float)
    f_time_in_heart_rate_zone_2 = Optional(float)
    f_time_in_heart_rate_zone_3 = Optional(float)
        

In [7]:
DB.commit() 

In [8]:
DB.generate_mapping()

In [ ]:
Partecipant.checkExistsDate(part='p11', date='2020-02-13 00:00:00')

In [ ]:
for d in date_range('2019-11-16','2020-03-13'):
    print(d)
    Partecipant.add_partecipant_day(name = "p11", day = d )

In [ ]:
vts  = datetime.strptime("2019-11-07", "%Y-%m-%d")
vte  = datetime.strptime("2019-11-08", "%Y-%m-%d")   #part, min_aggregation, vts, vte)
Partecipant.create_df_timeAggregation_Exercise(part = "p07", min_aggregation= 55, vts = vts, vte = vte)

In [ ]:
# Partecipant.add_partecipant_day(name = "p02", day = "2020-01-13" )
Partecipant.add_partecipant_day(name = "p11", day = "2019-11-17" )

In [ ]:
DB.commit()

In [ ]:
Partecipant.add_partecipant_day(name = "p07", day = "2019-11-01" )

In [ ]:
a = []
a.index(elem) if elem in a else -1

In [ ]:
DB.commit()

In [ ]:
date  = datetime.strptime("2019-11-16", "%Y-%m-%d")
Partecipant.createCalories(date = date, file = json_to_table("".join([PMDATA, 'p11', CALORIES])), name = 'p11')
# DB.commit()

In [ ]:

date  = datetime.strptime("2019-11-01", "%Y-%m-%d")
Partecipant.createBPM(date = date, file = json_to_table("".join([PMDATA, 'p01', HEART_RATE])), name = 'p01')
DB.commit()

In [ ]:

date  = datetime.strptime("2019-11-01", "%Y-%m-%d")
Partecipant.createDistances(date = date, file = json_to_table("".join([PMDATA, 'p01', DISTANCE])), name = 'p01')
DB.commit()

In [ ]:

date  = datetime.strptime("2019-11-01", "%Y-%m-%d")
Partecipant.createExercise(date = date, file = json_to_table("".join([PMDATA, 'p01', EXERCISE])), name = 'p01')
DB.commit()

In [ ]:

date  = datetime.strptime("2019-11-13", "%Y-%m-%d")
Partecipant.createSleep(date = date, file = json_to_table("".join([PMDATA, 'p09', SLEEP])), name = 'p09')
# DB.commit()

In [ ]:

date  = datetime.strptime("2019-11-01", "%Y-%m-%d")
Partecipant.createSteps(date = date, file = json_to_table("".join([PMDATA, 'p01', STEPS])), name = 'p01')
DB.commit()

In [ ]:

date  = datetime.strptime("2019-11-13", "%Y-%m-%d")
Partecipant.createDay(date = date, name = 'p09')
# DB.commit()

In [11]:
Partecipant.time_Aggregation(vts = "2020-01-05" , vte= "2020-01-06", minutes= 120)

12 2020-01-05 00:00:00 2020-01-06 00:00:00
['p07', 'p03', 'p01', 'p02', 'p04', 'p05', 'p11', 'p09']
Day for:  2020-01-05 00:00:00
True
2020-01-05 00:00:00  day not exist in db p07
Day for:  2020-01-05 00:00:00
Day for:  2020-01-05 00:00:00
Day for:  2020-01-05 00:00:00
Day for:  2020-01-05 00:00:00
Day for:  2020-01-05 00:00:00
Day for:  2020-01-05 00:00:00
True
2020-01-05 00:00:00  day not exist in db p11
Day for:  2020-01-05 00:00:00
Day for:  2020-01-06 00:00:00
Day for:  2020-01-06 00:00:00
Day for:  2020-01-06 00:00:00
Day for:  2020-01-06 00:00:00
Day for:  2020-01-06 00:00:00
Day for:  2020-01-06 00:00:00
Day for:  2020-01-06 00:00:00
True
2020-01-06 00:00:00  day not exist in db p11
Day for:  2020-01-06 00:00:00
['p07', 'p11', 'p11']
p2:  p07


ValueError: cannot convert float NaN to integer

In [ ]:

vts  = datetime.strptime("2020-02-13", "%Y-%m-%d")
vte  = datetime.strptime("2020-02-14", "%Y-%m-%d")   #part, min_aggregation, vts, vte)
Partecipant.create_df_timeAggregation_BPM2(part = "p11", min_aggregation = 360, vts = vts, vte = vte)

In [ ]:
vts  = datetime.strptime("2020-02-13", "%Y-%m-%d")
vte  = datetime.strptime("2020-02-14", "%Y-%m-%d")   #part, min_aggregation, vts, vte)
Partecipant.create_df_timeAggregation_Distances(part = "p11", min_aggregation = 120, vts = vts, vte = vte)

In [ ]:
vts  = datetime.strptime("2020-02-13", "%Y-%m-%d")
vte  = datetime.strptime("2020-02-14", "%Y-%m-%d")   #part, min_aggregation, vts, vte)
Partecipant.create_df_timeAggregation_Calories(part = "p11", min_aggregation = 120, vts = vts, vte = vte)

In [ ]:
vts  = datetime.strptime("2020-02-13", "%Y-%m-%d")
vte  = datetime.strptime("2020-02-14", "%Y-%m-%d")   #part, min_aggregation, vts, vte)
Partecipant.create_df_timeAggregation_Steps(part = "p11", min_aggregation = 120, vts = vts, vte = vte)

In [ ]:
vts  = datetime.strptime("2019-11-03", "%Y-%m-%d")
vte  = datetime.strptime("2019-11-09", "%Y-%m-%d")   #part, num_tuples, vts, vte)
Partecipant.create_df_timeAggregation_Exercise(part = "p07", num_tuples = 5, vts = vts, vte = vte)

In [ ]:
vts = "2019-11-01"
vts = datetime.strptime(vts, "%Y-%m-%d")
vte= "2019-11-08"
vte = datetime.strptime(vte, "%Y-%m-%d")
v = select((cal.vt, cal.value) 
                   for cal in Calories 
                   if (cal.vt >= vts and cal.vt <= vte and 'p01' == str(cal.partecipant.name))
                   ).order_by(1) 
v[:]

# v1 = Table(v, columns = ['vt', 'value'])
# v1

In [ ]:
file_path = 'D:\Alberto\Documenti\Documenti\UNIVR\Medical Bioinformatics\Secondo anno\Healthcare information systems\1.Data Warehouse OLAP systems for healthcare\datasets\pmdata\p09\fitbit'
date = "2019-11-13 00:00:00"
Partecipant.createCalories( date=date , file= file_path , name='p09')